In [1]:
import torch
import torch.nn as nn 
import pandas as pd
import numpy as np
import time
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import datetime

In [2]:
from src.algorithms.Transformer import TranformerAlgo
from src.utils.transformer_utils1 import *
from src.utils.transformer_utils2 import *
from src.dataset.Nabdataset import NabDataset
from src.model.modelTransformer import *

In [3]:
#This is the beginning of programm
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

12:31:10


<h4>Initializing Dataset</h4>

In [4]:
class ArgsTrn:
    workers=4
    batch_size=32
    epochs=5
    lr=0.0002
    cuda = True
    manualSeed=2
    mean=0
    std=0.1
    
opt_trn=ArgsTrn()
    
opt_trn=ArgsTrn()
torch.manual_seed(opt_trn.manualSeed)



class Datasettings:
    
    def __init__(self):
        '''
        end_name='ambient_temperature_system_failure.csv'
        self.dataset_name="NAB_Ambient_Temperature"
        key='realKnownCause/'+end_name 
        self.label_file = './lables/combined_windows.json'
        '''
        '''
        end_name='exchange-2_cpc_results.csv'
        self.dataset_name="exchange-2_cpc_results"
        key='realAdExchange/'+end_name 
        self.label_file = './lables/combined_windows.json'
        '''
        end_name='Twitter_volume_AAPL.csv'
        self.data_folder_path="/home/jupyter//GRANOGAN-IISC/data/nab/"
        key='realTweets/'+end_name 
        self.label_file = './lables/combined_windows.json'
        self.key=key
        self.train=True
        self.window_length=60

In [5]:
data_settings = Datasettings()
dataset = NabDataset(data_settings=data_settings)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=opt_trn.batch_size,
                                         shuffle=True, num_workers=int(opt_trn.workers))
seq_len = dataset.window_length # sequence length is equal to the window length
in_dim = dataset.n_feature # input dimension is same as number of feature
n_features=dataset.x.shape[2]
sequences=[x for i, (x,y) in enumerate(dataloader, 0)]

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [7]:
dataset.x.shape
embedding_dim=16

In [8]:
tranformerAlgo=TranformerAlgo(device=device,opt_trn=opt_trn,windows_length=seq_len,n_features=n_features,embedding_dim=embedding_dim)

In [9]:
trained_tranformer=tranformerAlgo.train_autoencoder(sequences)

RuntimeError: The size of tensor a (62) must match the size of tensor b (60) at non-singleton dimension 1

In [ ]:
PATH='/home/jupyter/src/saved_models/LSTMAutoencoder/yahoo/autoencoder.pkl'

In [ ]:
torch.save(trained_tranformer.module.state_dict(), PATH)

In [ ]:
state_dict = torch.load(PATH)
trained_tranformer=AnomalyTransformer(win_size=seq_len, enc_in=1, c_out=1, e_layers=3)
trained_tranformer=nn.DataParallel(trained_tranformer)
trained_tranformer.to(device)
trained_tranformer=lstmAutoencoderAlgo.load_model(state_dict,trained_tranformer)

<h4>Loading back the saved models</h4>

In [ ]:
tranformerAlgo.intialize_lstmautoencoder(trained_autoencoder)

<h2>Test Data</h2>


In [ ]:
class ArgsTest:
    workers = 1
    batch_size = 1
    
opt_test=ArgsTest()


class TestDataSettings:
    
    def __init__(self):
        '''
        end_name='exchange-2_cpc_results.csv'
        self.dataset_name="exchange-2_cpc_results"
        key='realAdExchange/'+end_name 
        self.label_file = './lables/combined_windows.json'
        '''
        '''
        end_name='ambient_temperature_system_failure.csv'
        self.dataset_name="NAB_Ambient_Temperature"
        key='realKnownCause/'+end_name 
        self.label_file = './lables/combined_windows.json'
        '''
        
        end_name='Twitter_volume_AAPL.csv'
        self.dataset_name="Twitter_volume_AAPL"
        key='realTweets/'+end_name 
        self.label_file = './lables/combined_windows.json'
        self.key=key
        self.train=False
        self.window_length=60        
        
test_data_settings = TestDataSettings()

In [ ]:
# define dataset object and data loader object in evaluation mood for NAB dataset
from mlutils import dataset
test_dataset = NabDataset(test_data_settings)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=opt_test.batch_size, 
                                         shuffle=False, num_workers=int(opt_test.workers))

test_dataset.x.shape, test_dataset.y.shape, test_dataset.data_len # check the dataset shape


test_sequences=[x for i, (x,y) in enumerate(test_dataloader, 0)]

In [ ]:
losses=tranformerAlgo.predict_loss(test_sequences)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(16,9), dpi=80)
plt.title('Loss Distribution', fontsize=16)
sns.histplot(losses, bins = 20, kde= False, color = 'blue');
#sns.distplot(losses, bins = 20, kde= True, color = 'blue');

In [ ]:
THRESHOLD =1.4

test_score_df = pd.DataFrame(index=range(len(losses)))
test_score_df['loss'] = [loss for loss in losses]
test_score_df['y'] = test_dataset.y
test_score_df['threshold'] = THRESHOLD
test_score_df['anomaly'] = test_score_df.loss > test_score_df.threshold
test_score_df['t'] = [x[59].item() for x in test_dataset.x]

plt.plot( test_score_df.loss, label='loss')
plt.plot( test_score_df.threshold, label='threshold')
#plt.plot( test_score_df.y, label='y')
plt.xticks(rotation=25)
plt.legend();

In [ ]:
import numpy as np
actual,predicted=improve_detection(test_score_df)

In [ ]:

from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_curve, auc, roc_auc_score

predicted = np.array(predicted)
actual = np.array(actual)

print_scores(predicted,actual)